In [8]:
# Standard Python libraries
import os
import time
from time import sleep
# Third-party libraries
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC

In [9]:
# Task 1: Log into LinkedIn
# Khởi tạo trình duyệt Chrome bằng Selenium
driver = webdriver.Chrome()  
sleep(3)  # Chờ trình duyệt khởi động

# URL trang đăng nhập LinkedIn
url = 'https://www.linkedin.com/login'  
driver.get(url)  # Truy cập trang LinkedIn

print('- Browser initialized successfully')  # In thông báo khởi tạo thành công
sleep(3)  # Chờ trang tải xong

# Đọc tên đăng nhập và mật khẩu từ file 'Login_info.txt'
with open('Login_info.txt') as credential:
    line = credential.readlines()  # Đọc các dòng từ file
    username = line[0].strip()  # Xóa khoảng trắng/dòng mới cho tên đăng nhập
    password = line[1].strip()  # Xóa khoảng trắng/dòng mới cho mật khẩu

print('- Login information successfully imported')  # In thông báo nhập thông tin thành công
sleep(2)  # Chờ trước bước tiếp theo

# Nhập tên đăng nhập và mật khẩu vào form đăng nhập
email_field = driver.find_element(By.ID, 'username')  # Tìm trường nhập email theo ID
email_field.send_keys(username)  # Nhập tên đăng nhập vào trường email
print('- Email entered successfully')  # In thông báo nhập email thành công
sleep(2)  # Chờ trước khi nhập mật khẩu

password_field = driver.find_element(By.NAME, 'session_password')  # Tìm trường nhập mật khẩu theo tên
password_field.send_keys(password)  # Nhập mật khẩu vào trường mật khẩu
print('- Password entered successfully')  # In thông báo nhập mật khẩu thành công
sleep(2)  # Chờ trước khi nhấn nút đăng nhập

# Nhấn nút đăng nhập
signin_field = driver.find_element(By.XPATH, '//button[@type="submit"]')  # Tìm nút gửi theo XPATH
signin_field.click()  # Nhấn nút đăng nhập
sleep(2)  # Chờ quá trình đăng nhập
print('- Task 1 completed: Logged into LinkedIn')  # In thông báo đăng nhập thành công

- Browser initialized successfully
- Login information successfully imported
- Email entered successfully
- Password entered successfully
- Task 1 completed: Logged into LinkedIn


In [11]:
# Task 2: Search for profiles to crawl

# Chờ cho thanh tìm kiếm có thể nhấp được
wait = WebDriverWait(driver, 10)  # Đặt thời gian chờ là 10 giây
search_field = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@placeholder="Search"]')))  # Chờ thanh tìm kiếm

# Nhập từ khóa tìm kiếm
search_query = input('Enter the keyword you want to search: ')  # Nhập từ khóa tìm kiếm từ người dùng
search_field.send_keys(search_query)  # Gửi từ khóa tìm kiếm tới LinkedIn

# Nhấn Enter để bắt đầu tìm kiếm
search_field.send_keys(Keys.RETURN)  # Sử dụng phím "Enter" để bắt đầu tìm kiếm
sleep(3)  # Chờ trang tải sau khi tìm kiếm

print('- Task 2 completed: Search performed')  # In thông báo tìm kiếm thành công


- Task 2 completed: Search performed


In [12]:
# Task 3: Filter search results and click 'People'

# Nhấn nút 'People' để lọc kết quả
signin_field = WebDriverWait(driver, 10).until(  # Chờ nút 'People' có thể nhấp được
    EC.presence_of_element_located((By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[3]/button'))  # Định vị nút 'People' theo XPATH
)
signin_field.click()  # Nhấn nút 'People' để lọc kết quả
sleep(2)  # Chờ trang cập nhật

print('- Task 3.0 completed: Clicked the People button')  # In thông báo nhấn nút People thành công


- Task 3.0 completed: Clicked the People button


In [13]:
# Task 4: Function to extract URLs from a profile page

def GetURL():
    # Phân tích HTML của trang hiện tại bằng BeautifulSoup
    page_source = BeautifulSoup(driver.page_source, 'html.parser')  
    profiles = page_source.find_all('a', class_='app-aware-link')  # Tìm tất cả các thẻ <a> có class 'app-aware-link'
    all_profile_URL = []  # Khởi tạo danh sách để lưu trữ URL profile

    for profile in profiles:  # Duyệt qua từng liên kết profile tìm thấy
        profile_URL = profile.get('href')  # Lấy thuộc tính 'href' (URL) từ thẻ <a>
        if profile_URL and "/in/" in profile_URL:  # Kiểm tra nếu URL chứa '/in/'
            if profile_URL not in all_profile_URL:  # Tránh trùng lặp
                all_profile_URL.append(profile_URL)  # Thêm URL vào danh sách

    return all_profile_URL  # Trả về danh sách URL profile

# Lấy số trang cần thu thập từ người dùng
input_page = int(input('How many profile pages do you want to scrape: '))  # Nhập số trang từ người dùng
URLs_all_page = []  # Khởi tạo danh sách để lưu trữ URL của tất cả các trang

for page in range(input_page):  # Duyệt qua từng trang để thu thập
    URLs_one_page = GetURL()  # Lấy URL từ trang hiện tại
    sleep(2)  # Chờ trước hành động tiếp theo
    
    # Cuộn đến cuối trang để tải thêm kết quả
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')  
    sleep(1)  # Chờ để tải nội dung mới
    
    # Nhấn nút "Next" để chuyển sang trang tiếp theo
    next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "artdeco-pagination__button--next"))  # Chờ nút 'Next' có thể nhấp được
    )
    driver.execute_script("arguments[0].click();", next_button)  # Nhấn nút 'Next'
    URLs_all_page += URLs_one_page  # Thêm URL từ trang này vào danh sách tổng
    sleep(4)  # Chờ trang tiếp theo tải xong

# In tất cả các URL đã thu thập
print('- Task 4 completed: Crawled URLs from all pages')  # In thông báo hoàn thành thu thập URL
for url in URLs_all_page:  # Duyệt qua tất cả các URL
    print(url)  # In từng URL


- Task 4 completed: Crawled URLs from all pages
https://www.linkedin.com/in/huynhvany?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADD20XEBYXyCenVcbWevuvFKBD8oE-wrtCs
https://www.linkedin.com/in/ACoAAD_xTccBlB-ME3ETdUw4iyfCVwEMPkiuRmY?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAD_xTccBlB-ME3ETdUw4iyfCVwEMPkiuRmY
https://www.linkedin.com/in/ACoAAFBrMBoBTvWFv9DTE4dEZF7fxuCwJjbq55Q?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAFBrMBoBTvWFv9DTE4dEZF7fxuCwJjbq55Q
https://www.linkedin.com/in/hoangtranne?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADS030MBop3v7MyqPtEcTXbKIrby5yHAqEQ
https://www.linkedin.com/in/ACoAABOZy7sBlPfXbF1y5PWn7V2jicfR_IPYi18?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABOZy7sBlPfXbF1y5PWn7V2jicfR_IPYi18
https://www.linkedin.com/in/khai180101?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAD8RlX4BqmfyQoXE2szC82Ga9FqBfjdicUM
https://www.linkedin.com/in/dinhtuann?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEh7OO0B8A3ZXT_wXmDHs4D_sTlCrAcRpHM
https://www.

In [7]:
# Task 5: Save data to an Excel file

# Đặt tên file Excel đầu ra
excel_filename = 'profile_data.xlsx'  

# Kiểm tra nếu file tồn tại và xóa nó để tránh ghi đè dữ liệu cũ
if os.path.exists(excel_filename):  # Kiểm tra nếu file tồn tại
    os.remove(excel_filename)  # Xóa file nếu nó tồn tại
    print(f'Deleted file: {excel_filename}')  # In thông báo xóa file
else:
    print(f'No file to delete: {excel_filename}')  # In thông báo nếu không tìm thấy file

data = []  # Danh sách để lưu trữ dữ liệu profile LinkedIn

# Duyệt qua từng URL trong danh sách URL đã thu thập
for linkedin_URL in URLs_all_page:  
    driver.get(linkedin_URL)  # Mở trang profile
    print('- Accessing profile: ', linkedin_URL)  # In thông báo đang truy cập profile
    time.sleep(2)  # Chờ trang tải xong

    # Trích xuất tên, chức danh và địa điểm
    page_source = BeautifulSoup(driver.page_source, "html.parser")  # Phân tích HTML của trang hiện tại

    name_elem = page_source.find('h1', class_='text-heading-xlarge')  # Tìm tên từ thẻ <h1>
    name = name_elem.get_text().strip() if name_elem else None  # Lấy tên hoặc None
    print('--- Profile name: ', name)  # In tên đã trích xuất

    title_elem = page_source.find('div', class_='text-body-medium break-words')  # Tìm chức danh
    title = title_elem.get_text().strip() if title_elem else None  # Lấy chức danh hoặc None
    print('--- Job title: ', title)  # In chức danh đã trích xuất

    location_elem = page_source.find('span', class_='text-body-small inline t-black--light break-words')  # Tìm địa điểm
    location = location_elem.get_text().strip() if location_elem else None  # Lấy địa điểm hoặc None
    print('--- Location: ', location)  # In địa điểm đã trích xuất

    # Lưu dữ liệu vào danh sách
    data.append({'Name': name, 'Title': title, 'Location': location})  # Lưu tên, chức danh và địa điểm vào danh sách

# Chuyển đổi danh sách dữ liệu thành DataFrame của pandas
df = pd.DataFrame(data)  # Tạo DataFrame từ dữ liệu

# Lưu DataFrame vào file Excel
df.to_excel(excel_filename, index=False)  # Ghi DataFrame vào file Excel mà không có chỉ số

print('Task completed! Data saved to Excel file.')  # In thông báo hoàn thành và lưu dữ liệu


Không có file nào cần xóa: output.xlsx
- Đang truy cập hồ sơ:  https://www.linkedin.com/in/huynhvany?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADD20XEBYXyCenVcbWevuvFKBD8oE-wrtCs
--- Tên hồ sơ là:  Y Huynh
--- Chức danh công việc là:  Data Engineer
--- Vị trí làm việc là:  Ho Chi Minh City, Vietnam


- Đang truy cập hồ sơ:  https://www.linkedin.com/in/ACoAAD_xTccBlB-ME3ETdUw4iyfCVwEMPkiuRmY?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAD_xTccBlB-ME3ETdUw4iyfCVwEMPkiuRmY
--- Tên hồ sơ là:  Quỳnh Trần
--- Chức danh công việc là:  Talent Acquisition at FPT Software
--- Vị trí làm việc là:  Da Nang City, Vietnam


- Đang truy cập hồ sơ:  https://www.linkedin.com/in/ACoAAFBrMBoBTvWFv9DTE4dEZF7fxuCwJjbq55Q?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAFBrMBoBTvWFv9DTE4dEZF7fxuCwJjbq55Q
--- Tên hồ sơ là:  Huy Bùi Quang
--- Chức danh công việc là:  ⚡ Data Engineer | Gen AI | LLMs | Database
--- Vị trí làm việc là:  Hanoi Capital Region


- Đang truy cập hồ sơ:  https://www.linkedin.co